# CPVO Register
El objetivo de este proyecto es el de buscar a través del buscador de la CVPO (Community Plant Variety Office) encargada de administrar a nivel europeo la propiedad intelectual de las distintas variedades de plantas. 

https://public.plantvarieties.eu/publicSearch



In [25]:
from selenium import webdriver
import chromedriver_binary
from time import strftime
import time
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import json


En primer lugar, descargaremos en un archivo la ficha de cada una de las variedades de la especie de interés. 

In [27]:
def obtenerURL_update(especie):
    """Elabora la url de la búsqueda adecuada en la web para descargar todas las variedades registradas hasta la fecha

        Parametros:
        especie (str): Nombre científico de la planta de interés
    
        Returns:
        url (str):url resultante sin el índice de página
    """
    str_specie="&speciesName="+especie.replace(" ", "%20")
    url="https://cpvonode.plantvarieties.eu/api/v2/publicSearch?"+str_specie+"&speciesNameFilter=contains&pageSize=100&pageNumber="
    return url

def descargarJSON(url, num_pag=1):
    """Descarga el contenido del archivo JSON correspondiente y lo devuelve en forma de diccionario
        Parametros:
        url(str): URL de la búsqueda en cuestión.
        num_page(int): Número de la página que se quiere descargar. Por defecto es la 1º página. 
        
        Returns:
        data(dict): Diccionario que contiene la información de la consulta.
    """
    result=[]
    respuesta = requests.get(str(url+str(num_pag)))
    data=respuesta.json()
    return data
    
def archivoUpdate(url, archivo="datos.json"):
    """Itera por todas las páginas de la base para descargar todos los datos y crear un archivo que los contenga todos
        
        Parametros:
        url(str): URL de la búsqueda en cuestión.
        archivo(str): Dirección del archivo que se quiere generar.  
        
    """
    
    data=descargarJSON(url) #descargamos la información de la 1º página
    
    #Determinamos cuantas páginas debemos iterar
    num_entradas=data['data']['count']
    iterador=int(num_entradas / 100) + (num_entradas % 100 > 0)
    
    result=[]
    i=2
    result.append(data['data']['registers'])
    print("Se ha registrado con éxito la página nº 1 de "+str(iterador))
    while i <= iterador:
        data=descargarJSON(url, i)
        result.append(data['data']['registers'])
        print("Se ha registrado con éxito la página nº: "+str(i)+" de "+str(iterador))
        i+=1
        time.sleep(1)
    with open(archivo, 'w') as json_file:
        json.dump(result, json_file)
    print("\n Su archivo ya se encuentra actualizado")

Poniendo de ejemplo el aceite de colza: 


In [28]:
nombre_cientifico="Brassica napus"
url=obtenerURL_update(nombre_cientifico)
archivoUpdate(url, nombre_cientifico+ ".json")

Se ha registrado con éxito la página nº 1 de 19
Se ha registrado con éxito la página nº: 2 de 19
Se ha registrado con éxito la página nº: 3 de 19
Se ha registrado con éxito la página nº: 4 de 19
Se ha registrado con éxito la página nº: 5 de 19
Se ha registrado con éxito la página nº: 6 de 19
Se ha registrado con éxito la página nº: 7 de 19
Se ha registrado con éxito la página nº: 8 de 19
Se ha registrado con éxito la página nº: 9 de 19
Se ha registrado con éxito la página nº: 10 de 19
Se ha registrado con éxito la página nº: 11 de 19
Se ha registrado con éxito la página nº: 12 de 19
Se ha registrado con éxito la página nº: 13 de 19
Se ha registrado con éxito la página nº: 14 de 19
Se ha registrado con éxito la página nº: 15 de 19
Se ha registrado con éxito la página nº: 16 de 19
Se ha registrado con éxito la página nº: 17 de 19
Se ha registrado con éxito la página nº: 18 de 19
Se ha registrado con éxito la página nº: 19 de 19

 Su archivo ya se encuentra actualizado


A continuación, definiremos una serie de funciones que nos permitan comprobar si ha habido alguna actualización, en cuyo caso generaremos una alerta

In [29]:
def actualizarAviso(url, archivo="datos.json"):
    """Itera por todas las páginas de la base para descargar todos los datos y crear un archivo que los contenga todos
        
        Parametros:
        url(str): URL de la búsqueda en cuestión.
        archivo(str): Dirección del archivo en que se guardan los datos.  
        
        Return:
        dif(int): Devuelve el número de entradas desactualizadas. En caso de dif=0, el archivo está actualizado. 
    """
    # Contamos el número de de entradas
    with open(archivo) as json_file:
        data_old = json.load(json_file)
    cont=0    
    for i in data_old:
        cont+=len(i)
    # Determinamos el número de entradas registradas en la web
    data_new=descargarJSON(url)
    dif=0
    if cont== data_new['data']['count']:
        print("El archivo sigue actualizado.")
    else:
        dif=data_new['data']['count']-cont
        print("El archivo no está actualizado. Hay un total de {0} entradas nuevas".format(dif))
    return dif


def imprimirAviso(url, archivo="datos.json"):
    """ Imprime en pantalla una serie de información pertinente, descarga el nuevo fichero actualizado e imprime por pantalla las entradas que se han añadido. 
        Parametros:
        url(str): URL de la búsqueda en cuestión.
        archivo(str): Dirección del archivo en que se guardan los datos.  
        
    """
    dif=actualizarAviso(url, archivo)
    if dif != 0:
        print("\n Procedemos a descargar los nuevos datos")
        archivoUpdate(url, archivo)
        with open(nombre_cientifico+ ".json") as json_file:
            data_actualizado = json.load(json_file)
        diccionario=data_actualizado[-1][-dif:]
        for i in diccionario:
            print("\n Entrada nueva:\n")
            print("Denominación: {0}\n Nombre de especie: {1}\n Día de solicitud: {2}".format(i['denomination'], i['speciesName'],i['applicationDate']))    

Comprobamos el funcionamiento. El archivo está actualizado ahora mismo, puesto que nos lo acabamos de descargar.

In [30]:
actualizarAviso(url, nombre_cientifico+ ".json")



El archivo sigue actualizado.


0

Vamos a retirar manualmente algunas entradas para similar una desactualización

In [31]:
with open(nombre_cientifico+ ".json") as json_file:
    data_desactualizado = json.load(json_file)

data_desactualizado[-1].pop()
data_desactualizado[-1].pop()
data_desactualizado[-1].pop()
data_desactualizado[-1].pop()

with open(nombre_cientifico+ ".json", 'w') as json_file:
    json.dump(data_desactualizado, json_file)


In [32]:
imprimirAviso(url, nombre_cientifico+ ".json")

El archivo no está actualizado. Hay un total de 4 entradas nuevas

 Procedemos a descargar los nuevos datos
Se ha registrado con éxito la página nº 1 de 19
Se ha registrado con éxito la página nº: 2 de 19
Se ha registrado con éxito la página nº: 3 de 19
Se ha registrado con éxito la página nº: 4 de 19
Se ha registrado con éxito la página nº: 5 de 19
Se ha registrado con éxito la página nº: 6 de 19
Se ha registrado con éxito la página nº: 7 de 19
Se ha registrado con éxito la página nº: 8 de 19
Se ha registrado con éxito la página nº: 9 de 19
Se ha registrado con éxito la página nº: 10 de 19
Se ha registrado con éxito la página nº: 11 de 19
Se ha registrado con éxito la página nº: 12 de 19
Se ha registrado con éxito la página nº: 13 de 19
Se ha registrado con éxito la página nº: 14 de 19
Se ha registrado con éxito la página nº: 15 de 19
Se ha registrado con éxito la página nº: 16 de 19
Se ha registrado con éxito la página nº: 17 de 19
Se ha registrado con éxito la página nº: 18 de 19
Se